<h1>Country Safety Index Building<h1>

## Dataset Building

#### Country GDP Data

In [217]:
import pandas as pd

# Country GDP Data
country_gdp_data = pd.read_csv("./data/country_gdp_data.csv")
gdp_2023 = country_gdp_data[["Country Name", "2023"]]
gdp_2023.rename(columns={"Country Name": "Country", "2023": "GDP 2023"}, inplace=True)

gdp_2023.head()


C:\Users\JasNj\AppData\Local\Temp\ipykernel_63092\1406084400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_2023.rename(columns={"Country Name": "Country", "2023": "GDP 2023"}, inplace=True)


,Country,GDP 2023
0,Aruba,3.648573e+09
1,Africa Eastern and Southern,1.245470e+12
2,Afghanistan,1.723305e+10
3,Africa Western and Central,7.991060e+11
4,Angola,8.482465e+10


#### Country Corruption, Burgalary and Theft Data

In [218]:
# Country Corruption, Burgalary and Theft Data
country_crime_data = pd.read_excel("./data/data_cts_corruption_and_economic_crime.xlsx")
crime_data_2021 = country_crime_data[["Country", "Category", "Year", "Unit of measurement", "VALUE", "Dimension", "Sex", "Age"]]


filtered = crime_data_2021[
    (crime_data_2021["Year"] == 2021) &
    (crime_data_2021["Unit of measurement"] == "Counts") &
    (crime_data_2021["Age"] == "Total") &
    (crime_data_2021["Sex"] == "Total") 
]

corruption_data_2021 = filtered[filtered["Category"] == "Corruption"][["Country", "VALUE"]].rename(columns={"VALUE": "Corruption"})
burglary_data_2021   = filtered[filtered["Category"] == "Burglary"][["Country", "VALUE"]].rename(columns={"VALUE": "Burglary"})
theft_data_2021      = filtered[filtered["Category"] == "Theft"][["Country", "VALUE"]].rename(columns={"VALUE": "Theft"})

print(corruption_data_2021.head())
print("")
print(burglary_data_2021.head())
print("")
print(theft_data_2021.head())


                  Country  Corruption
494               Albania       977.0
495  United Arab Emirates        84.0
496               Austria      2915.0
497            Azerbaijan       458.0
498               Belgium      4706.0

        Country  Burglary
4008    Albania     746.0
4009  Australia  139088.0
4010    Austria   40385.0
4011   Bulgaria    4134.0
4012    Bahamas      81.0

        Country     Theft
5893    Albania    3342.0
5894  Argentina  266920.0
5895  Australia  492210.0
5896    Austria   73213.0
5897   Bulgaria   24435.0


#### Intentional Homocide

In [219]:
# Intentional Homocide Data

homocide_data = pd.read_excel("./data/data_cts_intentional_homicide.xlsx")
homocide_data.head()

homocide_data_2021 = homocide_data[["Country", "Indicator", "Year", "Unit of measurement", "VALUE", "Dimension", "Sex", "Age", "Category"]]

filtered = homocide_data_2021[
    (homocide_data_2021["Year"] == 2021) &
    (homocide_data_2021["Unit of measurement"] == "Counts") &
    (homocide_data_2021["Indicator"] == "Victims of intentional homicide") &
    (homocide_data_2021["Dimension"] == "Total") &
    (homocide_data_2021["Age"] == "Total") &
    (homocide_data_2021["Sex"] == "Total") &
    (homocide_data_2021["Category"] == "Total")
]

homocide_data_2021 = filtered[["Country", "VALUE"]].rename(columns={"VALUE": "Homicide"})
homocide_data_2021.head()



,Country,Homicide
5441,Afghanistan,1613.0
5442,Albania,66.0
5443,United Arab Emirates,44.0
5444,Argentina,2094.0
5445,Armenia,61.0


#### Kidnappings, Assault, Sexual Violence

In [220]:
# Country Kidnappings, Assualt and Sexual Violence Data

country_assault_data = pd.read_excel("./data/data_cts_violent_and_sexual_crime.xlsx")
assault_data_2021 = country_assault_data[["Country", "Category", "Indicator", "Year", "Age", "Sex", "Unit of measurement", "VALUE"]]

filtered = assault_data_2021[
    (assault_data_2021["Year"] == 2021) &
    (assault_data_2021["Unit of measurement"] == "Counts") &
    (assault_data_2021["Indicator"] == "Violent offences") &
    (assault_data_2021["Age"] == "Total") &
    (assault_data_2021["Sex"] == "Total")
]

kidnappings_data     = filtered[filtered["Category"] == "Kidnapping"][["Country", "VALUE"]].rename(columns={"VALUE": "Kidnapping"})
assault_data         = filtered[filtered["Category"] == "Serious assault"][["Country", "VALUE"]].rename(columns={"VALUE": "Serious Assault"})
sexual_violence_data = filtered[filtered["Category"] == "Sexual violence"][["Country", "VALUE"]].rename(columns={"VALUE": "Sexual Violence"})

print(kidnappings_data.head())
print("")
print(assault_data.head())
print("")
print(sexual_violence_data.head())

                   Country  Kidnapping
3635               Albania         4.0
3636  United Arab Emirates       141.0
3637             Argentina       111.0
3638             Australia       453.0
3639               Austria         3.0

                   Country  Serious Assault
1796               Albania            163.0
1797  United Arab Emirates            145.0
1798             Argentina         154197.0
1799             Australia          74916.0
1800               Austria           3610.0

         Country  Sexual Violence
5398     Albania            135.0
5399   Argentina          37867.0
5400   Australia          31118.0
5401     Austria           4354.0
5402  Azerbaijan             77.0


#### Gun Ownership

In [221]:
# Gun Ownership Data
gun_ownership_data = pd.read_csv("./data/gun-ownership-by-country-2025.csv")

gun_ownership_data = gun_ownership_data[["country", "GunOwnership_TotalCivilianFirearms_num_2017", "GunOwnership_MilitaryFirearms_num_2017", "GunOwnership_LawEnforcementFirearms_num_2017"]]
gun_ownership_data.rename(columns={"country": "Country"}, inplace=True)
gun_ownership_data.head()




,Country,GunOwnership_TotalCivilianFirearms_num_2017,GunOwnership_MilitaryFirearms_num_2017,GunOwnership_LawEnforcementFirearms_num_2017
0,United States,393347000,4535380.0,1016000.0
1,Falkland Islands,2000,NaN,30.0
2,Yemen,14859000,28500.0,NaN
3,New Caledonia,115000,NaN,700.0
4,Serbia,2719000,384422.0,53100.0


#### Military Data

In [222]:
military_data = pd.read_csv("./data/military.csv")

military_data = military_data[["Country", "Total"]]
military_data.rename(columns={"Total": "Military"}, inplace=True)
military_data.head()

,Country,Military
0,Afghanistan,495000
1,Albania,8000
2,Algeria,467200
3,Angola,117000
4,Antigua and Barbuda,260


#### Natural Disaster Data

In [223]:
natural_disaster_data = pd.read_csv("./data/natural-disaster-risk-by-country-2025.csv")

natural_disaster_data = natural_disaster_data[["country", "NaturalDisasterRisk_WorldRiskIndexTotal_2024"]]
natural_disaster_data.rename(columns={"country": "Country"}, inplace=True)
natural_disaster_data.head()

,Country,NaturalDisasterRisk_WorldRiskIndexTotal_2024
0,Philippines,46.91
1,Indonesia,41.13
2,India,40.96
3,Colombia,37.81
4,Mexico,35.93


#### Police Data

In [224]:
police_data = pd.read_csv("./data/police_wikipedia_per_country.csv")

police_data = police_data[["Country or dependency", "Active force"]]
police_data.rename(columns={"Country or dependency": "Country"}, inplace=True)

police_data.head()

,Country,Active force
0,Algeria,160000
1,Albania,9900
2,American Samoa (US),200
3,Andorra,237
4,Antigua and Barbuda,600


#### Traffic Fatalities Data

In [225]:
traffic_data = pd.read_csv("./data/traffic_fatalities_per_100k.csv")

traffic_data = traffic_data[["GEO_NAME_SHORT", "RATE_PER_100000_N"]]

traffic_data.rename(columns={"GEO_NAME_SHORT": "Country", "RATE_PER_100000_N": "Traffic Fatalities per 100k"}, inplace=True)
traffic_data.head()

,Country,Traffic Fatalities per 100k
0,Tonga,8.5
1,Jamaica,17.8
2,Greece,7.3
3,Malawi,20.2
4,Bahrain,8.1


#### Youth Unemployement Data

In [226]:
youth_unemployment_data = pd.read_csv("./data/Youth unemployment rate (ages 15-24).csv")
youth_unemployment_data = youth_unemployment_data[["name", "%"]]
youth_unemployment_data.rename(columns={"name": "Country", "%": "Youth Unemployment Rate"}, inplace=True)

youth_unemployment_data.head()

,Country,Youth Unemployment Rate
0,Djibouti,76.5
1,Eswatini,65.0
2,Libya,49.4
3,South Africa,49.1
4,Kosovo,48.8


### Combining the data into one dataset

In [227]:
dfs = [
    gdp_2023,
    corruption_data_2021,
    burglary_data_2021,
    theft_data_2021,
    homocide_data_2021,
    kidnappings_data,
    assault_data,
    sexual_violence_data,
    gun_ownership_data,
    military_data,
    natural_disaster_data,
    police_data,
    traffic_data,
    youth_unemployment_data
]


combined_df = dfs[0]

for df in dfs[1:]:
    combined_df = pd.merge(combined_df, df, on="Country", how="outer")

#write to csv
combined_df.to_csv("country_safety_data.csv", index=False)
print("Combined DataFrame:")
combined_df.head()


Combined DataFrame:


,Country,GDP 2023,Corruption,Burglary,Theft,Homicide,Kidnapping,Serious Assault,Sexual Violence,GunOwnership_TotalCivilianFirearms_num_2017,GunOwnership_MilitaryFirearms_num_2017,GunOwnership_LawEnforcementFirearms_num_2017,Military,NaturalDisasterRisk_WorldRiskIndexTotal_2024,Active force,Traffic Fatalities per 100k,Youth Unemployment Rate
0,Aruba,3.648573e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,700.0,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,1.245470e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1.723305e+10,NaN,NaN,NaN,1613.0,NaN,NaN,NaN,4270000.0,331170.0,239000.0,495000.0,4.12,NaN,24.1,18.1
3,Africa Western and Central,7.991060e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,8.482465e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2982000.0,203300.0,60000.0,117000.0,10.42,NaN,15.0,28.2


## Data Cleaning

A big issue I noted was duplicate countries due to different naming conventions in the different csv files. Because there was too much data for me to handle manually, I got AI to generate the standardization mapping automatically, using a dictionary of common name variants and their corresponding standardized country names.

#### Function to standardize country names

In [228]:
import re

# get all the countries in the combined_df
countries = combined_df["Country"].unique().tolist()
print("Countries in the combined DataFrame:")
for country in countries:
    print(country)

# Standardize country names
standard_replacements = {
        # Korea variants
        "Republic of Korea": "South Korea",
        "Korea, South": "South Korea",
        "Korea, North": "North Korea",
        "Democratic People's Republic of Korea": "North Korea",
        "North Korea": "North Korea",
        "South Korea": "South Korea",

        # Moldova
        "Republic of Moldova": "Moldova",

        # Turkey
        "Türkiye": "Turkey",
        "Turkey (Turkiye)": "Turkey",

        # Czechia
        "Czech Republic": "Czechia",

        # Iran
        "Iran (Islamic Republic of)": "Iran",
        "Iran": "Iran",

        # Russia
        "Russian Federation": "Russia",
        "Russia": "Russia",

        # Venezuela
        "Venezuela (Bolivarian Republic of)": "Venezuela",
        "Venezuela, RB": "Venezuela",
        "Venezuela": "Venezuela",

        # Syria
        "Syrian Arab Republic": "Syria",
        "Syria": "Syria",

        # Palestine
        "occupied Palestinian territory, including east Jerusalem": "Palestine",
        "State of Palestine": "Palestine",
        "Palestinian Territory": "Palestine",
        "Palestine, State of": "Palestine",
        "West Bank": "Palestine",
        "Gaza Strip": "Palestine",
        "Palestine": "Palestine",

        # Hong Kong
        "China, Hong Kong SAR": "Hong Kong",
        "Hong Kong SAR, China": "Hong Kong",
        "Hong Kong (China)": "Hong Kong",
        "Hong Kong": "Hong Kong",

        # Macau
        "China, Macao SAR": "Macau",
        "Macao SAR, China": "Macau",
        "Macau (China)": "Macau",
        "Macau": "Macau",

        # China
        "People's Republic of China": "China",
        "China": "China",

        # Ivory Coast
        "Côte d'Ivoire": "Ivory Coast",
        "Ivory Coast": "Ivory Coast",

        # Cabo Verde
        "Cape Verde": "Cabo Verde",
        "Cabo Verde": "Cabo Verde",

        # Micronesia
        "Micronesia (Federated States of)": "Micronesia",
        "Micronesia, Federated States of": "Micronesia",
        "Micronesia": "Micronesia",

        # Myanmar
        "Burma": "Myanmar",
        "Myanmar": "Myanmar",

        # Vietnam
        "Viet Nam": "Vietnam",
        "Vietnam": "Vietnam",

        # United States
        "United States of America": "United States",
        "US": "United States",
        "United States": "United States",

        # United Kingdom
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "UK": "United Kingdom",
        "United Kingdom": "United Kingdom",
        "England and Wales": "United Kingdom",
        "Scotland": "United Kingdom",
        "Northern Ireland": "United Kingdom",

        # New Zealand
        "NZ": "New Zealand",
        "New Zealand": "New Zealand",

        # DR Congo / Republic of the Congo
        "DR Congo": "Democratic Republic of the Congo",
        "Congo, Dem. Rep.": "Democratic Republic of the Congo",
        "Congo, Democratic Republic of": "Democratic Republic of the Congo",
        "Democratic Republic of the Congo": "Democratic Republic of the Congo",

        "Congo, Republic of the": "Republic of the Congo",
        "Congo, Rep.": "Republic of the Congo",
        "Republic of the Congo": "Republic of the Congo",
        "Congo": "Republic of the Congo",

        # Gambia
        "Gambia, The": "Gambia",
        "Gambia": "Gambia",

        # Bahamas
        "Bahamas, The": "Bahamas",
        "Bahamas": "Bahamas",

        # Slovakia
        "Slovak Republic": "Slovakia",
        "Slovakia": "Slovakia",

        # Saint countries
        "St. Kitts and Nevis": "Saint Kitts and Nevis",
        "Saint Kitts and Nevis": "Saint Kitts and Nevis",
        "St. Lucia": "Saint Lucia",
        "Saint Lucia": "Saint Lucia",
        "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
        "Saint Vincent and the Grenadines": "Saint Vincent and the Grenadines",
        "Saint Barthelemy": "Saint Barthelemy",
        "Saint Martin": "Saint Martin",
        "St. Martin (French part)": "Saint Martin",

        # Territories
        "American Samoa (US)": "American Samoa",
        "Bermuda (UK)": "Bermuda",
        "Cayman Islands (UK)": "Cayman Islands",
        "Cook Islands (NZ)": "Cook Islands",
        "French Polynesia (France)": "French Polynesia",
        "Isle of Man (UK)": "Isle of Man",
        "Montserrat (UK)": "Montserrat",
        "New Caledonia (France)": "New Caledonia",
        "Niue (NZ)": "Niue",
        "Pitcairn Islands (UK)": "Pitcairn Islands",
        "Saint Helena (UK)": "Saint Helena",
        "Wallis and Futuna (France)": "Wallis and Futuna",
        "British Virgin Islands (UK)": "British Virgin Islands",
        "Sint Maarten (Netherlands)": "Sint Maarten",
        "Sint Maarten (Dutch part)": "Sint Maarten",
        "Virgin Islands (U.S.)": "United States Virgin Islands",
        "Virgin Islands": "United States Virgin Islands",

        # Holy See
        "Holy See": "Vatican City",
        "Vatican City": "Vatican City"
    }

def standardize_country_names(df, column="Country"):
    def clean_name(name):
        name = str(name).strip()

        # Extract parenthetical hint if present and treat as fallback country
        match = re.search(r"\(([^)]+)\)", name)
        if match:
            parenthetical = match.group(1).strip()
            # Try to match the parenthetical content first
            if parenthetical in standard_replacements:
                return standard_replacements[parenthetical]
            return parenthetical

        # Try to match the full name
        if name in standard_replacements:
            return standard_replacements[name]

        return name  # Return as-is if no match found

    df[column] = df[column].apply(clean_name)
    return df
    




Countries in the combined DataFrame:
Aruba
Africa Eastern and Southern
Afghanistan
Africa Western and Central
Angola
Albania
Andorra
Arab World
United Arab Emirates
Argentina
Armenia
American Samoa
Antigua and Barbuda
Australia
Austria
Azerbaijan
Burundi
Belgium
Benin
Burkina Faso
Bangladesh
Bulgaria
Bahrain
Bahamas, The
Bosnia and Herzegovina
Belarus
Belize
Bermuda
Bolivia
Brazil
Barbados
Brunei Darussalam
Bhutan
Botswana
Central African Republic
Canada
Central Europe and the Baltics
Switzerland
Channel Islands
Chile
China
Cote d'Ivoire
Cameroon
Congo, Dem. Rep.
Congo, Rep.
Colombia
Comoros
Cabo Verde
Costa Rica
Caribbean small states
Cuba
Curacao
Cayman Islands
Cyprus
Czechia
Germany
Djibouti
Dominica
Denmark
Dominican Republic
Algeria
East Asia & Pacific (excluding high income)
Early-demographic dividend
East Asia & Pacific
Europe & Central Asia (excluding high income)
Europe & Central Asia
Ecuador
Egypt, Arab Rep.
Euro area
Eritrea
Spain
Estonia
Ethiopia
European Union
Fragile and 

#### Combine Duplicates and Update the existing df

In [229]:
combined_df = standardize_country_names(combined_df)
merged_df = combined_df.groupby("Country", as_index=False).mean(numeric_only=True)
merged_df.to_csv("country_safety_data.csv", index=False)


#### Deleting Rows That Are Not Countries

In [230]:
import json

with open("./data/countries.json", "r") as f:
    countries = json.load(f)

valid_country_names = {c["name"] for c in countries}
merged_df = merged_df[merged_df["Country"].isin(valid_country_names)]
merged_df.to_csv("country_safety_data.csv", index=False)
merged_df.head()


,Country,GDP 2023,Corruption,Burglary,Theft,Homicide,Kidnapping,Serious Assault,Sexual Violence,GunOwnership_TotalCivilianFirearms_num_2017,GunOwnership_MilitaryFirearms_num_2017,GunOwnership_LawEnforcementFirearms_num_2017,Military,NaturalDisasterRisk_WorldRiskIndexTotal_2024,Active force,Traffic Fatalities per 100k,Youth Unemployment Rate
0,Afghanistan,1.723305e+10,NaN,NaN,NaN,1613.0,NaN,NaN,NaN,4270000.0,331170.0,239000.0,495000.0,4.12,NaN,24.1,18.1
4,Albania,2.354718e+10,977.0,746.0,3342.0,66.0,4.0,163.0,135.0,350000.0,21750.0,19000.0,8000.0,6.24,9900.0,10.8,28.2
5,Algeria,2.476260e+11,1579.0,NaN,NaN,695.0,239.0,10032.0,1812.0,877000.0,637720.0,363000.0,467200.0,9.64,160000.0,18.3,30.8
6,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,90.0,NaN,NaN,NaN,NaN,NaN
9,Angola,8.482465e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2982000.0,203300.0,60000.0,117000.0,10.42,NaN,15.0,28.2


#### Filling In Missing Data

In [231]:
merged_df.fillna(merged_df.mean(numeric_only=True), inplace=True)
merged_df.to_csv("country_safety_data.csv", index=False)
merged_df.head()

,Country,GDP 2023,Corruption,Burglary,Theft,Homicide,Kidnapping,Serious Assault,Sexual Violence,GunOwnership_TotalCivilianFirearms_num_2017,GunOwnership_MilitaryFirearms_num_2017,GunOwnership_LawEnforcementFirearms_num_2017,Military,NaturalDisasterRisk_WorldRiskIndexTotal_2024,Active force,Traffic Fatalities per 100k,Youth Unemployment Rate
0,Afghanistan,1.723305e+10,1999.191176,50125.595238,181453.848485,1613.000000,1415.965945,31491.712534,7791.105485,4270000.0,331170.000000,239000.0,495000.000000,4.12000,96902.100372,24.100000,18.100000
4,Albania,2.354718e+10,977.000000,746.000000,3342.000000,66.000000,4.000000,163.000000,135.000000,350000.0,21750.000000,19000.0,8000.000000,6.24000,9900.000000,10.800000,28.200000
5,Algeria,2.476260e+11,1579.000000,50125.595238,181453.848485,695.000000,239.000000,10032.000000,1812.000000,877000.0,637720.000000,363000.0,467200.000000,9.64000,160000.000000,18.300000,30.800000
6,American Samoa,5.860801e+11,1999.191176,50125.595238,181453.848485,2537.462046,1415.965945,31491.712534,7791.105485,400.0,555172.337838,90.0,248620.288591,8.26655,96902.100372,13.053488,15.939306
9,Angola,8.482465e+10,1999.191176,50125.595238,181453.848485,2537.462046,1415.965945,31491.712534,7791.105485,2982000.0,203300.000000,60000.0,117000.000000,10.42000,96902.100372,15.000000,28.200000
